# Expliquer les Préférences : Pourquoi une station est-elle plus prioritaire qu'une autre ?

**Question clé** : Comment expliquer à un décideur pourquoi le modèle considère qu'une station A est "plus prioritaire" qu'une station B pour les travaux de rénovation ?

Ce notebook propose une méthode pour **décomposer** cette comparaison en **arguments simples et compréhensibles** appelés *trade-offs* (compromis).

## Contexte RATP

La RATP doit prioriser les stations de métro pour des travaux de rénovation. Chaque station est évaluée selon plusieurs critères pondérés :
- **Affluence** (heures de pointe et creuses)
- **Priorité stratégique**
- **Niveau de dégradation**
- **Indice de connectivité**

In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from typing import List, Set, Optional, Dict
import warnings
warnings.filterwarnings('ignore')

## 1. Chargement des données

Le dataset RATP contient 10 stations de métro avec 7 critères :

| Critère | Description | Interprétation |
|---------|-------------|----------------|
| peak-entering-passengers/h | Passagers entrants en heure de pointe | Plus il y en a, plus la station est importante |
| peak-passing-passengers/h | Passagers en transit en heure de pointe | Indicateur de flux |
| off-peak-entering-passengers/h | Passagers entrants en heure creuse | Usage hors pointe |
| off-peak-passing-passengers/h | Passagers en transit en heure creuse | Flux hors pointe |
| strategic priority [0,10] | Priorité stratégique | Note attribuée par la direction |
| Station degradation level | Niveau de dégradation [0,20] | Plus c'est élevé, plus c'est dégradé |
| connectivity index [0,100] | Indice de connectivité | Correspondances et accessibilité |

In [ ]:
# Charger les données depuis Excel
df_raw = pd.read_excel('content/RATP.xlsx', header=None)

# Extraire les headers (ligne 2, index 2)
headers = df_raw.iloc[2, 1:].tolist()

# Extraire les données des stations (lignes 3-12)
df_stations = df_raw.iloc[3:13, 1:].copy()
df_stations.columns = headers
df_stations = df_stations.reset_index(drop=True)

# Extraire les poids (ligne 14, index 14)
weights_row = df_raw.iloc[14, 2:].values.astype(float)

# Colonne des noms de stations
station_col = 'Metro station'
station_names = df_stations[station_col].values

# Colonnes des features (tout sauf le nom)
feature_names = [c for c in df_stations.columns if c != station_col]

# Convertir en numérique
for col in feature_names:
    df_stations[col] = pd.to_numeric(df_stations[col], errors='coerce')

# Matrice X et vecteur weights
X = df_stations[feature_names].values.astype(float)
weights = weights_row.astype(float)

# Noms français pour l'affichage
noms_criteres = {
    'peak-entering-passengers/h': 'Entrants pointe (/h)',
    'peak-passing-passengers/h': 'Transit pointe (/h)',
    'off-peak-entering-passengers/h': 'Entrants creuse (/h)',
    'off-peak-passing-passengers/h': 'Transit creuse (/h)',
    'strategic priority [0,10]': 'Priorité stratégique',
    'Station degradation level ([0,20]  scale)': 'Dégradation station',
    'connectivity index [0,100]': 'Connectivité'
}

print(f"Dataset RATP : {len(X)} stations")
print(f"Nombre de critères : {len(feature_names)}")
print("\nCritères :")
for i, name in enumerate(feature_names):
    print(f"  {i+1}. {noms_criteres.get(name, name)}")

In [ ]:
# Afficher le tableau des stations avec leurs scores
df_display = df_stations.copy()
df_display['Score Priorité'] = X @ weights
df_display = df_display.sort_values('Score Priorité', ascending=False)
print("Classement des stations par score de priorité :")
print(df_display[[station_col, 'Score Priorité']].to_string(index=False))

## 2. Le modèle : Somme pondérée

On utilise une **somme pondérée** qui calcule un **score de priorité** pour chaque station :

$$\text{Score}(\text{station}) = \sum_{i=1}^{7} w_i \times \text{valeur}_i$$

- **Score élevé** → la station est plus prioritaire pour les travaux
- **Score bas** → la station est moins prioritaire

Les poids $w_i$ indiquent l'importance de chaque critère.

In [ ]:
print("Poids des critères (fournis dans le fichier) :")
print("-" * 50)
for name, w in zip(feature_names, weights):
    bar = '█' * int(w * 2)  # Barre proportionnelle au poids
    print(f"  {noms_criteres.get(name, name):<25} : {w:.4f} {bar}")

In [ ]:
EPS = 1e-6

def compute_omega(A, B, weights):
    """
    Calcule les contributions de chaque critère.
    omega_i = w_i * (A_i - B_i)
    sum(omega) = score(A) - score(B) > 0  <=>  A est plus prioritaire que B
    """
    return weights * (A - B)


def classify_omega(omega, eps=EPS):
    """Classifie les critères en pros, cons, neutral."""
    pros = [i for i, v in enumerate(omega) if v > eps]      # arguments POUR "A plus prioritaire"
    cons = [i for i, v in enumerate(omega) if v < -eps]     # arguments CONTRE
    neutral = [i for i, v in enumerate(omega) if abs(v) <= eps]
    return pros, cons, neutral

In [ ]:
def crit_name(i: int, feature_names: List[str]) -> str:
    """Retourne le nom français du critère."""
    return noms_criteres.get(feature_names[i], feature_names[i])

def fmt_q1_11(exp11, feature_names):
    """Formate une explication (1-1)."""
    lines = []
    for p, c in exp11:
        lines.append(f"- L'avantage sur {crit_name(p, feature_names)} compense l'inconvénient sur {crit_name(c, feature_names)}.")
    return "\n".join(lines) if lines else "- Dominance (aucun inconvénient)."

def fmt_q2_1m(exp1m, feature_names):
    """Formate une explication (1-m)."""
    lines = []
    for p, cons_list in exp1m:
        cons_names = ", ".join(crit_name(c, feature_names) for c in cons_list)
        lines.append(f"- L'avantage sur {crit_name(p, feature_names)} compense : {cons_names}.")
    return "\n".join(lines) if lines else "- Dominance (aucun inconvénient)."

def fmt_q3_m1(expm1, feature_names):
    """Formate une explication (m-1)."""
    lines = []
    for pros_list, c in expm1:
        pros_names = ", ".join(crit_name(p, feature_names) for p in pros_list)
        lines.append(f"- {pros_names} compensent l'inconvénient sur {crit_name(c, feature_names)}.")
    return "\n".join(lines) if lines else "- Dominance (aucun inconvénient)."

def fmt_q4(trade1m, tradem1, feature_names):
    """Formate une explication combinée."""
    lines = []
    if trade1m:
        lines.append("Trade-offs 1-m :")
        for p, cons_list in trade1m:
            cons_names = ", ".join(crit_name(c, feature_names) for c in cons_list)
            lines.append(f"  - {crit_name(p, feature_names)} compense : {cons_names}.")
    if tradem1:
        lines.append("Trade-offs m-1 :")
        for pros_list, c in tradem1:
            pros_names = ", ".join(crit_name(p, feature_names) for p in pros_list)
            lines.append(f"  - {pros_names} compensent : {crit_name(c, feature_names)}.")
    return "\n".join(lines) if lines else "- Dominance (aucun inconvénient)."

## 3. Comparer deux stations : le problème

Quand on compare deux stations A et B, on veut expliquer **pourquoi A est plus prioritaire que B** (ou inversement).

### Les trois types de critères

Pour chaque critère, on regarde la contribution au score :

- **Avantages (pros)** : critères où A est meilleure que B (contribution > 0)
- **Inconvénients (cons)** : critères où A est moins bien que B (contribution < 0)
- **Neutres** : critères où A et B sont égales

### L'idée des trade-offs

Pour expliquer que A est globalement plus prioritaire, on doit montrer comment ses **avantages compensent ses inconvénients**. C'est ce qu'on appelle un **trade-off** (compromis).

In [ ]:
class ComparaisonStations:
    """
    Compare deux stations (A vs B) et identifie les avantages/inconvénients.

    Convention RATP (priorité):
    - score(station) = sum_i w_i * valeur_i
    - Plus le score est élevé => station plus prioritaire
    - omega_i = w_i * (A_i - B_i)
      omega_i > 0 : argument POUR "A plus prioritaire que B"
      omega_i < 0 : argument CONTRE
    """

    def __init__(self, station_A: np.ndarray, station_B: np.ndarray,
                 weights: np.ndarray, feature_names: List[str]):
        self.A = station_A
        self.B = station_B
        self.weights = weights
        self.feature_names = feature_names

        # Scores de priorité (plus haut = plus prioritaire)
        self.score_A = float(np.dot(weights, station_A))
        self.score_B = float(np.dot(weights, station_B))

        # Contributions
        self.omega = compute_omega(station_A, station_B, weights)

        # Classification
        pros, cons, neutral = classify_omega(self.omega, eps=EPS)
        self.avantages = set(pros)
        self.inconvenients = set(cons)
        self.neutres = set(neutral)

        # A plus prioritaire ?
        self.difference = self.score_A - self.score_B
        self.A_plus_prioritaire = self.difference >= 0

    def afficher_comparaison(self, label_A="A", label_B="B"):
        """Affiche un résumé visuel de la comparaison."""
        print("\n" + "=" * 80)
        print(" COMPARAISON DE DEUX STATIONS (RATP)")
        print("=" * 80)

        print("\n Scores de priorité (plus haut = plus prioritaire) :")
        print(f"   Station {label_A} : {self.score_A:.2f}")
        print(f"   Station {label_B} : {self.score_B:.2f}")
        verdict = f"{label_A} est plus prioritaire" if self.A_plus_prioritaire else f"{label_B} est plus prioritaire"
        print(f"   \u2192 {verdict} (différence : {abs(self.difference):.2f})")

        print("\n Détail critère par critère :")
        print(f"{'Crit\u00e8re':<28} {label_A:>12} {label_B:>12} {'Omega':>12} {'Verdict':<15}")
        print("-" * 85)

        for i, name in enumerate(self.feature_names):
            diff = self.omega[i]
            if diff > EPS:
                verdict = "Avantage A"
            elif diff < -EPS:
                verdict = "Inconv. A"
            else:
                verdict = "Égalité"

            display_name = noms_criteres.get(name, name)[:27]
            print(f"{display_name:<28} {self.A[i]:>12.1f} {self.B[i]:>12.1f} {diff:>+12.2f} {verdict}")

        print("\n Résumé :")
        print(f"    {len(self.avantages)} avantage(s) pour A")
        print(f"    {len(self.inconvenients)} inconvénient(s) pour A")
        print(f"    {len(self.neutres)} critère(s) neutre(s)")

## 4. Les types d'explications (trade-offs)

Pour expliquer pourquoi A est plus prioritaire malgré certains inconvénients, on utilise trois types d'arguments :

### Type 1 : Échange simple (1-1)
"L'avantage sur le critère X compense l'inconvénient sur le critère Y"

*Exemple : "La station A a moins de passagers en transit, mais sa priorité stratégique est bien meilleure, ce qui compense."*

### Type 2 : Argument fort (1-m)
"Un seul avantage majeur compense plusieurs petits inconvénients"

*Exemple : "Bien que A soit légèrement moins bien sur la connectivité et le transit, son excellent niveau de dégradation compense tout."*

### Type 3 : Accumulation (m-1)
"Plusieurs petits avantages s'additionnent pour compenser un gros inconvénient"

*Exemple : "L'inconvénient majeur sur les passagers entrants est compensé par les avantages combinés sur la priorité et la connectivité."*

## 5. Algorithmes pour trouver les explications (solveurs Gurobi)

In [ ]:
def solve_explanation_11(omega, feature_names=None, eps=1e-6, verbose=False):
    """
    Q1 : Trouve une explication de type (1-1).
    Trade-off (1-1) : une paire (P, C) où un pro compense un con.
    """
    n = len(omega)
    if feature_names is None:
        feature_names = [f"C{i}" for i in range(n)]

    pros, cons, neutral = classify_omega(omega, eps=eps)
    
    if sum(omega) <= eps:
        return None, "INVALID"  # A n'est pas plus prioritaire que B
    if len(cons) == 0:
        return [], "TRIVIAL"    # Dominance
    if len(cons) > len(pros):
        return None, "INFEASIBLE"  # Plus de cons que de pros

    # Paires valides : omega[p] + omega[c] > 0
    valid_pairs = [(p, c) for p in pros for c in cons if omega[p] + omega[c] > eps]
    
    # Vérifier que chaque cons peut être couvert
    for c in cons:
        if not any((p, c) in valid_pairs for p in pros):
            return None, "INFEASIBLE"

    # Modèle Gurobi
    m = gp.Model("explain_11")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p, c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for (p, c) in valid_pairs}
    m.update()

    # C1 : Chaque cons couvert exactement 1 fois
    for c in cons:
        m.addConstr(gp.quicksum(z[p, c] for p in pros if (p, c) in z) == 1)

    # C2 : Chaque pro utilisé au plus 1 fois
    for p in pros:
        m.addConstr(gp.quicksum(z[p, c] for c in cons if (p, c) in z) <= 1)

    # Objectif : maximiser la marge totale
    m.setObjective(gp.quicksum(z[p, c] * (omega[p] + omega[c]) for (p, c) in valid_pairs), GRB.MAXIMIZE)

    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        explanation = [(p, c) for (p, c) in valid_pairs if z[p, c].X > 0.5]
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"


def solve_explanation_1m(omega, eps=1e-6, verbose=False):
    """
    Q2 : Trouve une explication de type (1-m).
    Trade-off (1-m) : un pro compense plusieurs cons.
    """
    pros, cons, neutral = classify_omega(omega, eps=eps)
    
    if sum(omega) <= eps:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"

    m = gp.Model("explain_1m")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p, c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for p in pros for c in cons}
    y = {p: m.addVar(vtype=GRB.BINARY, name=f"y_{p}") for p in pros}
    m.update()

    M = 10_000

    # C1 : Chaque cons assigné à exactement un pro
    for c in cons:
        m.addConstr(gp.quicksum(z[p, c] for p in pros) == 1)

    # C2 : Lien z-y
    for p in pros:
        for c in cons:
            m.addConstr(z[p, c] <= y[p])

    # C3 : Validité par pro utilisé
    for p in pros:
        m.addConstr(omega[p] + gp.quicksum(z[p, c] * omega[c] for c in cons) >= eps - M * (1 - y[p]))

    # Objectif : minimiser le nombre de trade-offs
    m.setObjective(gp.quicksum(y[p] for p in pros), GRB.MINIMIZE)

    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        explanation = []
        for p in pros:
            if y[p].X > 0.5:
                cons_for_p = [c for c in cons if z[p, c].X > 0.5]
                explanation.append((p, cons_for_p))
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"


def solve_explanation_m1(omega, eps=1e-6, verbose=False):
    """
    Q3 : Trouve une explication de type (m-1).
    Trade-off (m-1) : plusieurs pros compensent un seul cons.
    """
    pros, cons, neutral = classify_omega(omega, eps=eps)
    
    if sum(omega) <= eps:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"

    m = gp.Model("explain_m1")
    m.Params.OutputFlag = 1 if verbose else 0

    z = {(p, c): m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for p in pros for c in cons}
    m.update()

    # C1 : Chaque pro utilisé au plus 1 fois
    for p in pros:
        m.addConstr(gp.quicksum(z[p, c] for c in cons) <= 1)

    # C2 : Validité pour chaque cons
    for c in cons:
        m.addConstr(gp.quicksum(z[p, c] * omega[p] for p in pros) + omega[c] >= eps)

    # Objectif : minimiser le nombre de liens
    m.setObjective(gp.quicksum(z[p, c] for p in pros for c in cons), GRB.MINIMIZE)

    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        explanation = []
        for c in cons:
            pros_for_c = [p for p in pros if z[p, c].X > 0.5]
            explanation.append((pros_for_c, c))
        return explanation, "OPTIMAL"
    if m.status == GRB.INFEASIBLE:
        return None, "INFEASIBLE"
    return None, f"STATUS_{m.status}"


def solve_explanation_combined_q4(omega, feature_names=None, eps=1e-6, verbose=False):
    """
    Q4 (combinée) : explication mélangeant trade-offs (1-m) et (m-1).
    """
    n = len(omega)
    if feature_names is None:
        feature_names = [f"C{i}" for i in range(n)]

    pros = [i for i, v in enumerate(omega) if v > eps]
    cons = [i for i, v in enumerate(omega) if v < -eps]

    if float(sum(omega)) <= eps:
        return None, None, "INVALID"
    if len(cons) == 0:
        return [], [], "TRIVIAL"
    if len(pros) == 0:
        return None, None, "INFEASIBLE"

    model = gp.Model("explanation_combined_q4")
    model.Params.OutputFlag = 1 if verbose else 0

    M = 10_000.0

    z = {(p, c): model.addVar(vtype=GRB.BINARY, name=f"z_{p}_{c}") for p in pros for c in cons}
    mode1m = {c: model.addVar(vtype=GRB.BINARY, name=f"mode1m_{c}") for c in cons}
    y = {p: model.addVar(vtype=GRB.BINARY, name=f"y_{p}") for p in pros}
    w = {(p, c): model.addVar(vtype=GRB.BINARY, name=f"w_{p}_{c}") for p in pros for c in cons}
    has1m = {p: model.addVar(vtype=GRB.BINARY, name=f"has1m_{p}") for p in pros}

    model.update()

    # Lien z et y
    for p in pros:
        for c in cons:
            model.addConstr(z[p, c] <= y[p])

    # Chaque cons couvert par au moins un pro
    for c in cons:
        model.addConstr(gp.quicksum(z[p, c] for p in pros) >= 1)

    # Contrainte de mode
    for c in cons:
        nprosc = gp.quicksum(z[p, c] for p in pros)
        model.addConstr(nprosc <= 1 + M * (1 - mode1m[c]))
        model.addConstr(nprosc >= 2 - M * mode1m[c])

    # Validité par cons
    for c in cons:
        model.addConstr(gp.quicksum(z[p, c] * float(omega[p]) for p in pros) + float(omega[c]) >= eps)

    # Linéarisation w[p,c] = z[p,c] AND mode1m[c]
    for p in pros:
        for c in cons:
            model.addConstr(w[p, c] <= z[p, c])
            model.addConstr(w[p, c] <= mode1m[c])
            model.addConstr(w[p, c] >= z[p, c] + mode1m[c] - 1)

    # has1m[p]
    for p in pros:
        sumw = gp.quicksum(w[p, c] for c in cons)
        model.addConstr(has1m[p] <= sumw)
        model.addConstr(sumw <= M * has1m[p])

    # Validité 1-m par pro
    for p in pros:
        model.addConstr(float(omega[p]) + gp.quicksum(w[p, c] * float(omega[c]) for c in cons)
                        >= eps - M * (1 - has1m[p]))

    # Exclusivité
    for p in pros:
        for c in cons:
            model.addConstr(gp.quicksum(z[p, c2] for c2 in cons)
                            <= 1 + M * (1 - z[p, c]) + M * mode1m[c])

    # Objectif : minimiser le nombre de liens
    model.setObjective(gp.quicksum(z[p, c] for p in pros for c in cons), GRB.MINIMIZE)

    model.optimize()

    if model.status == GRB.INFEASIBLE:
        return None, None, "INFEASIBLE"
    if model.status != GRB.OPTIMAL:
        return None, None, f"STATUS_{model.status}"

    # Construire l'explication
    cons_in_1m = [c for c in cons if mode1m[c].X > 0.5]
    cons_in_m1 = [c for c in cons if mode1m[c].X <= 0.5]

    proto_cons = {}
    for c in cons_in_1m:
        for p in pros:
            if z[p, c].X > 0.5:
                proto_cons.setdefault(p, []).append(c)

    tradeoffs_1m = [(p, proto_cons[p]) for p in proto_cons]

    tradeoffs_m1 = []
    for c in cons_in_m1:
        pros_for_c = [p for p in pros if z[p, c].X > 0.5]
        tradeoffs_m1.append((pros_for_c, c))

    return tradeoffs_1m, tradeoffs_m1, "OPTIMAL"

## 6. Exemples concrets

Comparons des stations réelles du dataset pour illustrer les explications.

In [ ]:
def analyser_et_expliquer(idx_A: int, idx_B: int, label_A: str = None, label_B: str = None):
    """
    Analyse complète de deux stations avec explication.
    """
    comp = ComparaisonStations(X[idx_A], X[idx_B], weights, feature_names)

    if label_A is None:
        label_A = station_names[idx_A]
    if label_B is None:
        label_B = station_names[idx_B]

    print(f"\n{'='*80}")
    print(f"Comparaison : {label_A} vs {label_B}")

    comp.afficher_comparaison(label_A=label_A, label_B=label_B)

    if comp.A_plus_prioritaire:
        omega = comp.omega
        print("\n" + "-" * 80)
        print(" RECHERCHE D'EXPLICATION")
        print("-" * 80)

        # Essayer Q1 (1-1)
        exp11, st11 = solve_explanation_11(omega, feature_names=feature_names, verbose=False)
        if st11 in ["OPTIMAL", "TRIVIAL"]:
            print(f"\n>>> Explication (1-1) trouvée ! [Status: {st11}]")
            print(fmt_q1_11(exp11, feature_names))
            return comp

        # Essayer Q2 (1-m)
        exp1m, st1m = solve_explanation_1m(omega, verbose=False)
        if st1m in ["OPTIMAL", "TRIVIAL"]:
            print(f"\n>>> Explication (1-m) trouvée ! [Status: {st1m}]")
            print(fmt_q2_1m(exp1m, feature_names))
            return comp

        # Essayer Q3 (m-1)
        expm1, stm1 = solve_explanation_m1(omega, verbose=False)
        if stm1 in ["OPTIMAL", "TRIVIAL"]:
            print(f"\n>>> Explication (m-1) trouvée ! [Status: {stm1}]")
            print(fmt_q3_m1(expm1, feature_names))
            return comp

        # Essayer Q4 (combinée)
        trade1m, tradem1, st4 = solve_explanation_combined_q4(omega, feature_names=feature_names, verbose=False)
        if st4 in ["OPTIMAL", "TRIVIAL"]:
            print(f"\n>>> Explication combinée (Q4) trouvée ! [Status: {st4}]")
            print(fmt_q4(trade1m, tradem1, feature_names))
            return comp

        print("\n>>> Aucune explication trouvée (même avec Q4).")
    else:
        print(f"\nAttention : {label_B} est en fait plus prioritaire que {label_A} (selon le score).")
    
    return comp

In [ ]:
# Calculer les scores de priorité pour toutes les stations
scores_priorite = X @ weights

# Trouver la station la plus et la moins prioritaire
idx_plus_prioritaire = int(np.argmax(scores_priorite))
idx_moins_prioritaire = int(np.argmin(scores_priorite))

print("="*80)
print("Exemple 1 : Station la plus prioritaire vs la moins prioritaire")
print("="*80)
comp1 = analyser_et_expliquer(idx_plus_prioritaire, idx_moins_prioritaire)

In [ ]:
# Exemple 2 : Deux stations proches dans le classement
ordre = np.argsort(-scores_priorite)  # Tri décroissant
i, j = int(ordre[0]), int(ordre[1])   # Les deux plus prioritaires

print("\n" + "="*80)
print("Exemple 2 : Les deux stations les plus prioritaires")
print("="*80)
comp2 = analyser_et_expliquer(i, j)

In [ ]:
# Exemple 3 : Chercher un cas avec plusieurs inconvénients (trade-offs intéressants)
np.random.seed(42)
trouve = False
n = len(X)

for _ in range(500):
    i, j = np.random.choice(n, 2, replace=False)
    if scores_priorite[i] > scores_priorite[j]:
        comp_test = ComparaisonStations(X[i], X[j], weights, feature_names)
        if comp_test.A_plus_prioritaire and len(comp_test.inconvenients) >= 2:
            print("\n" + "="*80)
            print("Exemple 3 : Cas avec plusieurs compromis")
            print("="*80)
            comp3 = analyser_et_expliquer(i, j)
            trouve = True
            break

if not trouve:
    print("Pas de cas avec plusieurs compromis trouvé dans l'échantillon.")

## 7. Statistiques d'explicabilité

Quelle proportion des comparaisons peut-on expliquer avec chaque type de trade-off ?

In [ ]:
def calculer_statistiques():
    """
    Calcule les statistiques d'explicabilité pour toutes les paires de stations.
    """
    n = len(X)
    
    # Construire toutes les paires où A est plus prioritaire que B
    paires = []
    for i in range(n):
        for j in range(n):
            if i != j and scores_priorite[i] > scores_priorite[j]:
                paires.append((i, j))

    stats = {
        "total": len(paires),
        "dominance": 0,
        "q1_11": 0,
        "q2_1m": 0,
        "q3_m1": 0,
        "q4_combined": 0,
        "non_explicable": 0
    }

    for i, j in paires:
        comp = ComparaisonStations(X[i], X[j], weights, feature_names)

        # Dominance = pas d'inconvénients
        if len(comp.inconvenients) == 0:
            stats["dominance"] += 1
            continue

        omega = comp.omega

        # Q1 (1-1)
        exp11, st11 = solve_explanation_11(omega, feature_names=feature_names, verbose=False)
        if st11 == "OPTIMAL":
            stats["q1_11"] += 1
            continue

        # Q2 (1-m)
        exp1m, st1m = solve_explanation_1m(omega, verbose=False)
        if st1m == "OPTIMAL":
            stats["q2_1m"] += 1
            continue

        # Q3 (m-1)
        expm1, stm1 = solve_explanation_m1(omega, verbose=False)
        if stm1 == "OPTIMAL":
            stats["q3_m1"] += 1
            continue

        # Q4 (combinée)
        trade1m, tradem1, st4 = solve_explanation_combined_q4(omega, feature_names=feature_names, verbose=False)
        if st4 == "OPTIMAL":
            stats["q4_combined"] += 1
            continue
        
        stats["non_explicable"] += 1

    return stats


print(" Calcul des statistiques d'explicabilité...")
stats = calculer_statistiques()

explicable = stats["total"] - stats["non_explicable"]

print(f"\n{'='*65}")
print("STATISTIQUES D'EXPLICABILITÉ - DATASET RATP")
print(f"{'='*65}")
print(f"\nSur {stats['total']} comparaisons analysées :")
print("\n{:<35} {:>10} {:>10}".format("Type d'explication", "Nombre", "%"))
print("-" * 57)
print(f"{'Dominance (aucun compromis)':<35} {stats['dominance']:>10} {100*stats['dominance']/stats['total']:>9.1f}%")
print(f"{'Q1 : 1-1 (échange simple)':<35} {stats['q1_11']:>10} {100*stats['q1_11']/stats['total']:>9.1f}%")
print(f"{'Q2 : 1-m (argument fort)':<35} {stats['q2_1m']:>10} {100*stats['q2_1m']/stats['total']:>9.1f}%")
print(f"{'Q3 : m-1 (accumulation)':<35} {stats['q3_m1']:>10} {100*stats['q3_m1']/stats['total']:>9.1f}%")
print(f"{'Q4 : combinée':<35} {stats['q4_combined']:>10} {100*stats['q4_combined']/stats['total']:>9.1f}%")
print("-" * 57)
print(f"{'TOTAL EXPLICABLE':<35} {explicable:>10} {100*explicable/stats['total']:>9.1f}%")
print(f"{'Non explicable':<35} {stats['non_explicable']:>10} {100*stats['non_explicable']/stats['total']:>9.1f}%")

## 8. Résumé : Toutes les comparaisons du classement

In [ ]:
# Classement par score de priorité
ordre = np.argsort(-scores_priorite)
ranking = [station_names[i] for i in ordre]

print("="*80)
print("RÉSUMÉ : Types d'explications pour les comparaisons consécutives")
print("="*80)

results = []

for k in range(len(ordre) - 1):
    i, j = ordre[k], ordre[k + 1]
    c1, c2 = station_names[i], station_names[j]
    
    comp = ComparaisonStations(X[i], X[j], weights, feature_names)
    omega = comp.omega

    # Test des différents types
    if len(comp.inconvenients) == 0:
        status_dom = 'OK'
    else:
        status_dom = '-'
    
    _, st11 = solve_explanation_11(omega, feature_names=feature_names, verbose=False)
    _, st1m = solve_explanation_1m(omega, verbose=False)
    _, stm1 = solve_explanation_m1(omega, verbose=False)
    _, _, st4 = solve_explanation_combined_q4(omega, feature_names=feature_names, verbose=False)

    results.append({
        'Rang': f"{k+1} > {k+2}",
        'Comparaison': f"{c1[:20]} > {c2[:20]}",
        'Dom': status_dom,
        '(1-1)': 'OK' if st11 == 'OPTIMAL' else '-',
        '(1-m)': 'OK' if st1m == 'OPTIMAL' else '-',
        '(m-1)': 'OK' if stm1 == 'OPTIMAL' else '-',
        'Comb': 'OK' if st4 == 'OPTIMAL' else '-'
    })

df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

In [ ]:
# Afficher le classement complet
print("\n" + "="*60)
print("CLASSEMENT COMPLET DES STATIONS PAR PRIORITÉ")
print("="*60)
for k, i in enumerate(ordre):
    print(f"{k+1:>2}. {station_names[i]:<40} (Score: {scores_priorite[i]:.2f})")